In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, brier_score_loss, f1_score, fbeta_score, confusion_matrix
from tabpfn import TabPFNClassifier
from imblearn.over_sampling import SMOTENC
from concurrent.futures import ThreadPoolExecutor

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

np.random.seed(99)

# Bootstrap function with ThreadPoolExecutor
def bootstrap_metrics(y_true, y_prob, n_bootstraps=1000, random_state=42, max_workers=60):
    np.random.seed(random_state)
    metrics_list = []
    
    def compute_metrics(indices):
        y_true_boot = y_true.iloc[indices]
        y_prob_boot = y_prob[indices]
        return evaluate_single_bootstrap(y_true_boot, y_prob_boot)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _ in range(n_bootstraps):
            indices = np.random.choice(len(y_true), len(y_true), replace=True)
            futures.append(executor.submit(compute_metrics, indices))
        
        for future in futures:
            metrics_list.append(future.result())
    
    # Aggregate results
    metrics_df = pd.DataFrame(metrics_list)
    mean_metrics = metrics_df.mean()
    ci_lower = metrics_df.quantile(0.025)
    ci_upper = metrics_df.quantile(0.975)
    
    return pd.DataFrame({
        'Mean': mean_metrics,
        'CI Lower': ci_lower,
        'CI Upper': ci_upper
    })

df1 = pd.read_csv('~/data/BAH_PRS/version10/ml_dat/conpass.csv',sep=',',header=0)

set1 = ['Age', 'BMI', 'SBP', 'DBP', 'PAC', 'Renin', 'Sex']
set2 = ['Age', 'BMI', 'WC', 'TG', 'LDL', 'FBG', 'SBP', 'DBP', 'Renin', 'Sex', 'ASCVD']
set3 = ['Age', 'BMI', 'WC', 'TG', 'LDL', 'FBG', 'SBP', 'DBP', 'Sex', 'ASCVD']

X = df1[set2]
y = df1['IHA']

smotenc = SMOTENC(categorical_features=[9,10], 
                  random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, 
                                                    random_state=42)
X_train_resampled, y_train_resampled = smotenc.fit_resample(X_train, y_train)
X_train_scaled = X_train_resampled
X_test_scaled = X_test
scaler = StandardScaler()
X_train_scaled.iloc[:,0:9] = scaler.fit_transform(X_train_scaled.iloc[:,0:9])
X_test_scaled.iloc[:,0:9] = scaler.transform(X_test_scaled.iloc[:,0:9])

/tmp/ipykernel_456912/2633986274.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.92718868  1.24146073  1.15574473 ...  0.89859675  0.12715279
 -1.24430312]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train_scaled.iloc[:,0:9] = scaler.fit_transform(X_train_scaled.iloc[:,0:9])
/tmp/ipykernel_456912/2633986274.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.15858713  0.47001677 -0.12999519  0.72716476 -1.5871671   1.92718868
 -0.0442792   0.55573277  1.6700407   0.0414368  -0.47285917 -0.12999519
  0.38430078 -0.21571118 -0.90143914 -2.95862302  1.24146073 -1.50145111
  1.15574473  1.07002874  1.32717672  0.72716476  0.12715279 -0.64429116
 -1.41573511 -0.55857516  1.24146073 -0.55857516  0.98431274 -0.30142718
  0.38430078 -1.5871671   0.38430078 -1.50145111 -2.35861105 -1.50145111

In [2]:
# Define evaluation metrics
def evaluate_single_bootstrap(y_true_boot, y_prob_boot):
    y_pred_boot = (y_prob_boot > 0.5).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true_boot, y_pred_boot).ravel()
    
    return {
        'AUC': roc_auc_score(y_true_boot, y_prob_boot),
        'Accuracy': accuracy_score(y_true_boot, y_pred_boot),
        'Sensitivity': tp / (tp + fn) if (tp + fn) != 0 else 0,
        'Specificity': tn / (tn + fp) if (tn + fp) != 0 else 0,
        'PPV': tp / (tp + fp) if (tp + fp) != 0 else 0,
        'NPV': tn / (tn + fn) if (tn + fn) != 0 else 0,
        'LR+': (tp / (tp + fn)) / (1 - (tn / (tn + fp))) if (tn + fp) != 0 and (tp + fn) != 0 else float('inf'),
        'LR-': (1 - (tp / (tp + fn))) / (tn / (tn + fp)) if (tn + fp) != 0 and (tp + fn) != 0 else float('inf'),
        'DOR': ((tp / (tp + fn)) / (1 - (tn / (tn + fp)))) / ((1 - (tp / (tp + fn))) / (tn / (tn + fp)))
                if (tn + fp) != 0 and (tp + fn) != 0 else float('inf'),
        'F1': f1_score(y_true_boot, y_pred_boot),
        'F2': fbeta_score(y_true_boot, y_pred_boot, beta=2),
        'Brier Score': brier_score_loss(y_true_boot, y_prob_boot)
    }
    
# Train TabPFN model
model = TabPFNClassifier(n_estimators=32, device='cpu', random_state=42, softmax_temperature=0.6, balance_probabilities=True)
model.fit(X_train_scaled, y_train_resampled)

# Evaluate on the test set
y_prob = model.predict_proba(X_test_scaled)[:, 1]
results_df1 = bootstrap_metrics(y_test, y_prob)
results_df1
# Save probabilities
pd.DataFrame(y_prob, columns=['Probability']).to_csv('/home/luo_wenjin/data/BAH_PRS/version10/ml_dat/prob_model2.csv', index=False)

TabPFNClassifier(balance_probabilities=True, device='cpu', n_estimators=32,
                 random_state=42, softmax_temperature=0.6)

,Mean,CI Lower,CI Upper
AUC,0.864354,0.824492,0.899406
Accuracy,0.849292,0.817460,0.876984
Sensitivity,0.302396,0.205463,0.400000
Specificity,0.961913,0.942854,0.978873
PPV,0.620970,0.461486,0.763158
NPV,0.870053,0.838771,0.897162
LR+,8.529386,4.517439,15.677344
LR-,0.725309,0.621415,0.827289
DOR,11.996015,5.548310,23.393212
F1,0.404904,0.294482,0.510949


In [3]:
# Evaluate on validation sets
validation_sets = [
    pd.read_csv('~/data/BAH_PRS/version10/ml_dat/pato.csv',sep=',',header=0),
    pd.read_csv('~/data/BAH_PRS/version10/ml_dat/monash.csv',sep=',',header=0),
    pd.read_csv('~/data/BAH_PRS/version10/ml_dat/ljubljana.csv',sep=',',header=0)
]

def evaluate_single_validation_set(df, model, scaler):
    X_val = df[set2]
    y_val = df['IHA']
    X_val.iloc[:, 0:9] = scaler.fit_transform(X_val.iloc[:, 0:9])  
    
    y_prob = model.predict_proba(X_val)[:, 1]
    return bootstrap_metrics(y_val, y_prob)

def evaluate_on_validation_sets(model, validation_sets, scaler):
    results = {}
    with ThreadPoolExecutor() as executor:
        futures = {
            f'Validation Set {i+2}': executor.submit(evaluate_single_validation_set, df, model, scaler)
            for i, df in enumerate(validation_sets)
        }
        for key, future in futures.items():
            results[key] = future.result()
    return results

# Evaluate on validation sets
results_df2 = evaluate_on_validation_sets(model, validation_sets, scaler)
results_df2

/tmp/ipykernel_456912/728613764.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.98698266 -0.80089916 -1.17729534  0.61058649  0.89288362 -1.08319629
  0.70468553 -0.51860203  0.32828936 -0.42450299 -0.98909725  0.89288362
 -0.04810681 -0.42450299 -0.14220586 -0.33040394 -1.17729534 -1.17729534
 -2.30648386  0.51648745 -1.55369151 -0.80089916  0.23419032  0.98698266
  1.08108171 -1.45959247  1.17518075 -2.11828577 -0.33040394  0.70468553
  0.51648745 -2.4005829  -0.89499821 -1.08319629 -0.04810681 -0.14220586
  0.23419032  0.61058649  0.70468553  0.14009127 -1.08319629  0.70468553
 -0.70680012 -0.14220586 -0.33040394  0.14009127  0.61058649 -0.33040394
 -0.51860203 -1.27139438  1.17518075 -0.42450299  0.51648745  0.04599223
 -0.42450299  1.36337884  0.32828936 -0.14220586  0.32828936  0.32828936
  1.17518075 -1.08319629  1.17518075 -0.51860203  0.79878458  0.98698266
 -0.70680012  0.4223884   1.08108171  0.

{'Validation Set 2':                  Mean  CI Lower  CI Upper
 AUC          0.725779  0.672009  0.775053
 Accuracy     0.874119  0.858277  0.888821
 Sensitivity  0.218851  0.129253  0.320761
 Specificity  0.900785  0.886132  0.915082
 PPV          0.082311  0.044188  0.123436
 NPV          0.965915  0.956224  0.974675
 LR+          2.218923  1.264509  3.342768
 LR-          0.867258  0.755201  0.970940
 DOR          2.610352  1.300224  4.391295
 F1           0.119183  0.065421  0.175007
 F2           0.163589  0.093333  0.235565
 Brier Score  0.091743  0.082648  0.100733,
 'Validation Set 3':                  Mean  CI Lower  CI Upper
 AUC          0.727498  0.685123  0.769549
 Accuracy     0.652182  0.612844  0.688119
 Sensitivity  0.232102  0.176160  0.291084
 Specificity  0.919002  0.890241  0.948496
 PPV          0.645437  0.538433  0.753256
 NPV          0.653244  0.610870  0.694470
 LR+          2.977613  1.866768  4.685881
 LR-          0.835819  0.768288  0.904467
 DOR         

In [4]:
results_df1.to_csv('/home/luo_wenjin/data/BAH_PRS/version10/ml_dat/conpass_training_boot_model2.csv', index=True)
for key, value in results_df2.items():
    value.to_csv(f'/home/luo_wenjin/data/BAH_PRS/version10/ml_dat/{key}_boot_model2.csv', index=True)

In [5]:
X = df1[set3]
y = df1['IHA']

smotenc = SMOTENC(categorical_features=[8,9], 
                  random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, 
                                                    random_state=42)
X_train_resampled, y_train_resampled = smotenc.fit_resample(X_train, y_train)
X_train_scaled = X_train_resampled
X_test_scaled = X_test
scaler = StandardScaler()
X_train_scaled.iloc[:,0:8] = scaler.fit_transform(X_train_scaled.iloc[:,0:8])
X_test_scaled.iloc[:,0:8] = scaler.transform(X_test_scaled.iloc[:,0:8])

model = TabPFNClassifier(n_estimators=32, device='cpu', random_state=42, softmax_temperature=0.6, balance_probabilities=True)
model.fit(X_train_scaled, y_train_resampled)

# Evaluate on the test set
y_prob = model.predict_proba(X_test_scaled)[:, 1]
results_df1 = bootstrap_metrics(y_test, y_prob)
results_df1
# Save probabilities
pd.DataFrame(y_prob, columns=['Probability']).to_csv('/home/luo_wenjin/data/BAH_PRS/version10/ml_dat/prob_model3.csv', index=False)

/tmp/ipykernel_456912/2810011555.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.92946149  1.2437939   1.15808545 ...  0.90096011  0.12958407
 -0.81320887]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train_scaled.iloc[:,0:8] = scaler.fit_transform(X_train_scaled.iloc[:,0:8])
/tmp/ipykernel_456912/2810011555.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.15604266  0.47241786 -0.12754128  0.72954321 -1.58458491  1.92946149
 -0.04183283  0.55812631  1.67233615  0.04387562 -0.47037507 -0.12754128
  0.38670942 -0.21324973 -0.89891732 -2.95592009  1.2437939  -1.49887646
  1.15808545  1.07237701  1.32950235  0.72954321  0.12958407 -0.64179197
 -1.41316801 -0.55608352  1.2437939  -0.55608352  0.98666856 -0.29895817
  0.38670942 -1.58458491  0.38670942 -1.49887646 -2.35596094 -1.49887646

TabPFNClassifier(balance_probabilities=True, device='cpu', n_estimators=32,
                 random_state=42, softmax_temperature=0.6)

,Mean,CI Lower,CI Upper
AUC,0.697751,0.633860,0.754083
Accuracy,0.817431,0.783730,0.845238
Sensitivity,0.174067,0.097819,0.255821
Specificity,0.949912,0.928073,0.969938
PPV,0.417836,0.256250,0.600000
NPV,0.848139,0.815445,0.876575
LR+,3.667345,1.766227,6.574180
LR-,0.869611,0.782127,0.953385
DOR,4.289567,1.851390,8.508141
F1,0.244246,0.143971,0.340451


In [6]:
def evaluate_single_validation_set(df, model, scaler):
    X_val = df[set3]
    y_val = df['IHA']
    X_val.iloc[:, 0:8] = scaler.fit_transform(X_val.iloc[:, 0:8])  
    
    y_prob = model.predict_proba(X_val)[:, 1]
    return bootstrap_metrics(y_val, y_prob)

# Evaluate on validation sets
results_df2 = evaluate_on_validation_sets(model, validation_sets, scaler)

results_df1.to_csv('~/data/BAH_PRS/version10/ml_dat/conpass_training_boot_model3.csv', index=True)
for key, value in results_df2.items():
    value.to_csv(f'~/data/BAH_PRS/version10/ml_dat/{key}_boot_model3.csv', index=True)

/tmp/ipykernel_456912/2476877814.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.98698266 -0.80089916 -1.17729534  0.61058649  0.89288362 -1.08319629
  0.70468553 -0.51860203  0.32828936 -0.42450299 -0.98909725  0.89288362
 -0.04810681 -0.42450299 -0.14220586 -0.33040394 -1.17729534 -1.17729534
 -2.30648386  0.51648745 -1.55369151 -0.80089916  0.23419032  0.98698266
  1.08108171 -1.45959247  1.17518075 -2.11828577 -0.33040394  0.70468553
  0.51648745 -2.4005829  -0.89499821 -1.08319629 -0.04810681 -0.14220586
  0.23419032  0.61058649  0.70468553  0.14009127 -1.08319629  0.70468553
 -0.70680012 -0.14220586 -0.33040394  0.14009127  0.61058649 -0.33040394
 -0.51860203 -1.27139438  1.17518075 -0.42450299  0.51648745  0.04599223
 -0.42450299  1.36337884  0.32828936 -0.14220586  0.32828936  0.32828936
  1.17518075 -1.08319629  1.17518075 -0.51860203  0.79878458  0.98698266
 -0.70680012  0.4223884   1.08108171  0.